### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn1 import NN1
from mlcvlab.models.nn2 import NN2
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.adam import Adam

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    return x, y


def prepare_data(x, y):
    y = y.astype(np.int)
    y = np.where(y%2==0, 1, 0)
    
    #y = np.apply_along_axis(lambda num: num % 2 == 0, 0, y).astype("int")
    return x, y


def split_train_test(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=1000, random_state=42)
    return X_train, X_test, y_train, y_test


def initialize_model(X_train, X_test, y_train, y_test):
    hidden_neuron_count = 200
    t = np.sqrt(3.0 * (1/max(1., (2+2)/2.)))
    W0 = np.random.uniform(-t, t, size= (hidden_neuron_count, 784))
    W1 = np.random.uniform(-t, t, size= (1, hidden_neuron_count))
    
    
    print(f"Size of W0 : {W0.shape}, Size of W1 : {W1.shape}")
    two_layer_nn = NN2()
    two_layer_nn.W = [W0, W1]
    two_layer_nn.layers[0].W = W0
    two_layer_nn.layers[1].W = W1

    return two_layer_nn


def train_model(model, X_train, y_train):
    #final_W = Adam(model, X_train, y_train)
    final_W = SGD(model, X_train, y_train, lr=0.1)
    return final_W


def test_model(model, X_test, y_test, final_W):
    model.W = final_W

    y_hat = model.nn2(X_test.T).T
    y_test = y_test.reshape(-1, 1)
    error = np.round_(np.abs(y_test - y_hat), 4)

    indicator_fn = np.greater(error, np.zeros(error.shape)).astype('float')

    accuracy = np.mean(indicator_fn) * 100
    return accuracy

### 5. Run the program

In [6]:

#load data
x, y = load_dataset()
print("Dataset import successfully")

#prepare data
x, y = prepare_data(x,y)
print("Data preprocessing successfully")

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)
print("Dataset Splitted successfully")

#initialize model
model = initialize_model(X_train, X_test, y_train, y_test)
print("Model initalized successfully")

#training model
final_W = train_model(model, X_train, y_train)
print(f"Completed training model - final W : {final_W}")
print("Model Trained successfully")


#testing model
accuracy = test_model(model, X_test, y_test, final_W)
print(f"Completed testing model - Accuracy : {accuracy}")

Dataset import successfully
Data preprocessing successfully
Dataset Splitted successfully
Size of W0 : (200, 784), Size of W1 : (1, 200)
Model initalized successfully
Completed training model - final W : [array([[ 2.60972144e-04,  6.25573077e-04,  8.74009973e-05, ...,
         5.98775438e-04, -1.03476161e-04,  3.49241311e-04],
       [-1.59578868e-04,  1.04340901e-04, -1.93867376e-04, ...,
        -6.28891953e-05,  3.02456997e-04,  7.05766959e-04],
       [ 2.03522324e-04, -8.46911187e-05,  2.60230281e-05, ...,
         7.41246384e-04,  1.30423048e-04,  3.79607081e-04],
       ...,
       [ 3.74933818e-04,  6.26763852e-04,  7.47515547e-04, ...,
         7.95861332e-04,  1.93108581e-04,  6.67412029e-05],
       [ 6.50188967e-04,  2.66122537e-04,  6.34171312e-04, ...,
         2.82743770e-04,  1.74506666e-04,  6.11938280e-06],
       [-1.11146834e-04,  5.32708134e-04,  1.81260199e-04, ...,
         2.18847553e-04,  2.19343349e-04,  4.36314074e-04]]), array([[2.46861767e-03, 7.37042476e-0